In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".20"

In [2]:
import pandas as pd
from model import Brain
from submodels import factories
import matplotlib.pyplot as plt
import pandas as pd
from jf.db import DB
from lib.plot import plot_2d_positions_colors
from lib.sde.grn2 import GRNMain2
from lib.sde.grn import GRNMain, grn_from_json
from lib.sde.cell2 import Cell2, CellBatch
from lib.sde.mutate import mutate_grn2
from jf.profiler import Profiler

In [3]:
from itertools import accumulate
import numpy as np
from collections import defaultdict
import jax.numpy as jnp
import numpy as np

In [4]:
HISTORY = defaultdict(dict)

In [5]:
END_TIME = 54

In [6]:
HALL_OF_FAME = []

In [7]:
REF = pd.read_csv("output/results/setup_basic/export/ref_basic.csv")  # ref is a mean

In [8]:
def individual_generator():
    return Solution(GRNMain2(5, 0, 5))

In [9]:
# set a similar grn
ref_grn = """
{"object_name": "GRNMain2", "nb_start_gene": 5, "nb_off_trees": 0, "nb_genes": 5,
"params": [[2.007115364074707, 0.3403625786304474, 1.3039988279342651, 2.0225210189819336, 1.5461641550064087],
[2.375922203063965, 7.7354350090026855, 5.84783411026001, 6.466616153717041, 8.159892082214355],
[7.479958534240723, 5.624154090881348, 8.094839096069336, 1.19766366481781, 8.925475120544434],
[9.56630802154541, 3.818467378616333, 9.031471252441406, 7.2058916091918945, 1.283474326133728],
[6.358775615692139, 5.765143394470215, 9.488754272460938, 2.5621650218963623, 5.002293586730957],
[3.9804110527038574, 4.080349922180176, 9.770094871520996, 9.090896606445312, 4.010365009307861],
[5.9285149574279785, 3.148050546646118, 4.948235511779785, 3.5156848430633545, 4.3454718589782715],
[8.12299919128418, 2.998713493347168, 5.796504020690918, 9.65429973602295, 0.45337215065956116]],
"trees": [{"op": "Var", "val": 2}, {"op": "Var", "val": 0},
{"op": "And", "F1": {"op": "Or", "F1": {"op": "Var", "val": 1},
"F2": {"op": "Or", "F1": {"op": "Var", "val": 2}, "F2": {"op": "Var", "val": 3}}},
"F2": {"op": "Not", "F": {"op": "Or", "F1": {"op": "Var", "val": 4}, "F2": {"op": "Var", "val": 0}}}},
{"op": "And", "F1": {"op": "Var", "val": 4}, "F2": {"op": "Var", "val": 2}},
{"op": "Or", "F1": {"op": "Var", "val": 2}, "F2": {"op": "Not", "F": {"op": "Or", "F1": {"op": "Var", "val": 1},
"F2": {"op": "Var", "val": 3}}}}]}
"""

In [10]:
# set a similar grn
ref_grn = """
{"object_name": "GRNMain2", "nb_start_gene": 5, "nb_off_trees": 0, "nb_genes": 5,
"params": [[2.007115364074707, 0.3403625786304474, 1.3039988279342651, 2.0225210189819336, 1.5461641550064087],
[0.1, 0.1, 0.1, 0.1, 0.1],
[7.479958534240723, 5.624154090881348, 8.094839096069336, 1.19766366481781, 8.925475120544434],
[9.56630802154541, 3.818467378616333, 9.031471252441406, 7.2058916091918945, 1.283474326133728],
[6.358775615692139, 5.765143394470215, 9.488754272460938, 2.5621650218963623, 5.002293586730957],
[3.9804110527038574, 4.080349922180176, 9.770094871520996, 9.090896606445312, 4.010365009307861],
[5.9285149574279785, 3.148050546646118, 4.948235511779785, 3.5156848430633545, 4.3454718589782715],
[8.12299919128418, 2.998713493347168, 5.796504020690918, 9.65429973602295, 0.45337215065956116]],
"trees": [{"op": "Var", "val": 2}, {"op": "Var", "val": 0},
{"op": "And", "F1": {"op": "Or", "F1": {"op": "Var", "val": 1},
"F2": {"op": "Or", "F1": {"op": "Var", "val": 2}, "F2": {"op": "Var", "val": 3}}},
"F2": {"op": "Not", "F": {"op": "Or", "F1": {"op": "Var", "val": 4}, "F2": {"op": "Var", "val": 0}}}},
{"op": "And", "F1": {"op": "Var", "val": 4}, "F2": {"op": "Var", "val": 2}},
{"op": "Or", "F1": {"op": "Var", "val": 2}, "F2": {"op": "Not", "F": {"op": "Or", "F1": {"op": "Var", "val": 1},
"F2": {"op": "Var", "val": 3}}}}]}
"""

In [11]:
def config_numpy():
    grn = GRNMain(5, 0, 5)
    ccls = factories["grn1"](grn=grn)

    bb = Brain(time_step=0.5, verbose=False, start_population=4, max_pop_size=1e3,
            cell_cls=ccls.generate, end_time=60, start_time=50, silent=True)
    bb.run()

In [12]:
def config_jax1():
    grn = grn_from_json(ref_grn)
    grn.compile()
    ccls = factories["grn2"](grn=grn)

    bb = Brain(time_step=0.5, verbose=False, start_population=4, max_pop_size=1e3,
            cell_cls=ccls, end_time=60, start_time=50, silent=False)
    bb.run()
    return grn

In [13]:
# grn = config_jax1()

In [14]:
# grn.to_json()

In [15]:
def config_jax2():
    grn = grn_from_json(ref_grn)
    grn.compile()
    ccls = factories["grn2_opti"](grn=grn)

    bb = Brain(time_step=0.5, verbose=False, start_population=4, max_pop_size=1e3,
            cell_cls=ccls, end_time=60, start_time=50, silent=False)
    bb.run()
    return grn

In [20]:
%timeit grn = config_jax2()

Ticking abs : 50.0, step : 0.5, size : 16
Ticking abs : 50.5, step : 0.5, size : 12
Ticking abs : 51.0, step : 0.5, size : 16
Ticking abs : 51.5, step : 0.5, size : 19
Ticking abs : 52.0, step : 0.5, size : 25
Ticking abs : 52.5, step : 0.5, size : 30
Ticking abs : 53.0, step : 0.5, size : 42
Ticking abs : 53.5, step : 0.5, size : 51
Ticking abs : 54.0, step : 0.5, size : 69
Ticking abs : 54.5, step : 0.5, size : 89
Ticking abs : 55.0, step : 0.5, size : 117
Ticking abs : 55.5, step : 0.5, size : 144
Ticking abs : 56.0, step : 0.5, size : 192
Ticking abs : 56.5, step : 0.5, size : 244
Ticking abs : 57.0, step : 0.5, size : 320
Ticking abs : 57.5, step : 0.5, size : 419
Ticking abs : 58.0, step : 0.5, size : 558
Ticking abs : 58.5, step : 0.5, size : 726
Ticking abs : 59.0, step : 0.5, size : 957
Ticking abs : 59.5, step : 0.5, size : 1251
Population explosion
Population exploded or extinguished
Ticking abs : 50.0, step : 0.5, size : 16
Ticking abs : 50.5, step : 0.5, size : 5
Populatio

In [17]:
def config_jax3():
    grn = grn_from_json(ref_grn)
    grn.compile()
    ccls = factories["grn2_opti"](grn=grn)

    bb = Brain(time_step=0.5, verbose=False, start_population=4, max_pop_size=1e3,
            cell_cls=ccls, end_time=60, start_time=50, silent=False, opti=True)
    bb.run()
    return grn

In [21]:
%timeit grn = config_jax3()

Ticking abs : 50.0, step : 0.5, size : 16
Ticking abs : 50.5, step : 0.5, size : 12
Ticking abs : 51.0, step : 0.5, size : 13
Ticking abs : 51.5, step : 0.5, size : 20
Ticking abs : 52.0, step : 0.5, size : 23
Ticking abs : 52.5, step : 0.5, size : 25
Ticking abs : 53.0, step : 0.5, size : 38
Ticking abs : 53.5, step : 0.5, size : 45
Ticking abs : 54.0, step : 0.5, size : 55
Ticking abs : 54.5, step : 0.5, size : 80
Ticking abs : 55.0, step : 0.5, size : 99
Ticking abs : 55.5, step : 0.5, size : 128
Ticking abs : 56.0, step : 0.5, size : 174
Ticking abs : 56.5, step : 0.5, size : 217
Ticking abs : 57.0, step : 0.5, size : 292
Ticking abs : 57.5, step : 0.5, size : 380
Ticking abs : 58.0, step : 0.5, size : 508
Ticking abs : 58.5, step : 0.5, size : 669
Ticking abs : 59.0, step : 0.5, size : 880
Ticking abs : 59.5, step : 0.5, size : 1167
Population explosion
Population exploded or extinguished
Ticking abs : 50.0, step : 0.5, size : 16
Ticking abs : 50.5, step : 0.5, size : 10
Ticking a

It seems that jax jits functions according to the size of the input.  
Therefore when the size is changed, it requires to recompute everything

# Comparison non opti and opti for computations

First we create Cell2 with already defined grn and init val (with set_quant / expr / deriv)
Test run

In [19]:
raise

RuntimeError: No active exception to reraise

In [ ]:
grn = grn_from_json(ref_grn)
grn.compile()

In [ ]:
grn.noise

In [ ]:
vec_quant = grn.get_vector_quantities()

In [ ]:
c = Cell2(grn, vec_quant)

In [ ]:
c.run_step()

In [ ]:
c.quantities

In [ ]:
c.check_action(1, 1), c.check_action(1, 2)

In [ ]:
def compute_mean_raw(vec_quant, grn):
    cells = []
    for i in range(100):
        c = Cell2(grn, vec_quant)
        c.run_step()
        c.run_step()
        cells.append(c)
    mean_q = np.var(np.array([c.quantities for c in cells]), axis=0)
    mean_e = np.var(np.array([c.expression for c in cells]), axis=0)
    mean_d = np.var(np.array([c.derivative for c in cells]), axis=0)
    return mean_q, mean_e, mean_d

In [ ]:
def compute_mean_batch(vec_quant, grn):
    cells = [Cell2(grn, vec_quant) for i in range(100)]
    b = CellBatch(cells)
    b.run_step()
    b.run_step()
    mean_q = np.var(np.array([c.quantities for c in b.cells]), axis=0)
    mean_e = np.var(np.array([c.expression for c in b.cells]), axis=0)
    mean_d = np.var(np.array([c.derivative for c in b.cells]), axis=0)
    return mean_q, mean_e, mean_d

In [ ]:
res1 = compute_mean_raw(vec_quant, grn)
print("e")

In [ ]:
res2 = compute_mean_batch(vec_quant, grn)
print("e")

In [ ]:
for i, j in zip(res1, res2):
    print("Compare ->")
    print(i)
    print(j)
    print()